## Imports


In [ ]:
# imports
import os
import shutil
import random
import kaggle
import torch
from torchvision import transforms
from torchvision import datasets
from torchvision import models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np


## Prepare data

In [ ]:
# paths
tmp_path = 'tmp'
train_dataset_path = f'{tmp_path}/train_dataset'
test_dataset_path = f'{tmp_path}/test_dataset'


In [ ]:
# download dataset
if not os.path.exists(train_dataset_path) or not os.path.exists(test_dataset_path):
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('grassknoted/asl-alphabet', path=tmp_path, unzip=True)

    shutil.move(f"{tmp_path}/asl_alphabet_train/asl_alphabet_train", train_dataset_path)
    shutil.move(f"{tmp_path}/asl_alphabet_test/asl_alphabet_test", test_dataset_path)

    shutil.rmtree(f"{tmp_path}/asl_alphabet_train")
    shutil.rmtree(f"{tmp_path}/asl_alphabet_test")


## Configure PyTorch


In [ ]:
# PyTorch device
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


## Prepare data


In [ ]:
# Define transform
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
transform


In [ ]:
# Load dataset
dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
dataset


In [ ]:
# Get classes
classes = dataset.classes
classes


In [ ]:
# Count classes
classes_count = len(classes)
classes_count


In [ ]:
# Set train/test split ratio
train_ratio = 0.8
test_ratio = 0.2
{"train_ratio": train_ratio, "test_ratio": test_ratio}


In [ ]:
# Set train/test sizes
train_size = int(train_ratio * len(dataset))
test_size = int(test_ratio * len(dataset))
{"train_size": train_size, "test_size": test_size}


In [ ]:
# Split dataset into train and test
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


In [ ]:
# Count train/test dataset sizes
train_dataset_count = len(train_dataset)
test_dataset_count = len(test_dataset)
{"train_dataset_count": train_dataset_count, "test_dataset_count": test_dataset_count}


In [ ]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)


In [ ]:
# Display random images function
def show_random_samples(dataset, classes, num_samples=5):
    fig, axs = plt.subplots(1, num_samples, figsize=(15, 15))

    for i in range(num_samples):
        index = np.random.randint(0, len(dataset) - 1)
        image, label = dataset[index]

        axs[i].imshow(image.permute(1, 2, 0))
        axs[i].set_title(classes[label])
        axs[i].axis('off')

    plt.show()


In [ ]:
# Display random images from train dataset
show_random_samples(train_dataset, classes)


In [ ]:
# Display random images from test dataset
show_random_samples(test_dataset, classes)


## Prepare models


### Make models folder

In [ ]:
os.makedirs('tmp/models', exist_ok=True)


### AlexNet


In [ ]:
AlexNet_model = models.alexnet()

# Freeze model parameters
for param in AlexNet_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
AlexNet_model.classifier[6] = torch.nn.Linear(AlexNet_model.classifier[6].in_features, classes_count)

# Move the model to the device (GPU if available)
AlexNet_model.to(device)

# Define loss function and optimizer
AlexNet_model_criterion = torch.nn.CrossEntropyLoss()
AlexNet_model_optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=0.001)


In [ ]:
summary(AlexNet_model, input_size=(3, 200, 200))


### GoogleNet


In [ ]:
GoogleNet_model = models.googlenet(models.GoogLeNet_Weights.DEFAULT)

# Freeze model parameters
for param in GoogleNet_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
GoogleNet_model.fc = torch.nn.Linear(GoogleNet_model.fc.in_features, classes_count)

# Move the model to the device (GPU if available)
GoogleNet_model.to(device)

# Define loss function and optimizer
GoogleNet_model_criterion = torch.nn.CrossEntropyLoss()
GoogleNet_model_optimizer = torch.optim.Adam(GoogleNet_model.parameters(), lr=0.001)


In [ ]:
summary(GoogleNet_model, input_size=(3, 200, 200))


### MobileNet V2


In [ ]:
# MobileNetV2_model = models.MobileNetV2()

# # Freeze model parameters
# for param in MobileNetV2_model.parameters():
#     param.requires_grad = False

# # Modify the last fully connected layer to fit your number of classes
# MobileNetV2_model.classifier[6] = torch.nn.Linear(MobileNetV2_model.classifier[6].in_features, classes_count)

# # Move the model to the device (GPU if available)
# MobileNetV2_model.to(device)

# # Define loss function and optimizer
# MobileNetV2_model_criterion = torch.nn.CrossEntropyLoss()
# MobileNetV2_model_optimizer = torch.optim.Adam(MobileNetV2_model.parameters(), lr=0.001)


In [ ]:
# summary(MobileNetV2_model, input_size=(3, 200, 200))


### MobileNet V3


In [ ]:
# MobileNetV3_model = models.MobileNetV3()

# # Freeze model parameters
# for param in MobileNetV3_model.parameters():
#     param.requires_grad = False

# # Modify the last fully connected layer to fit your number of classes
# MobileNetV3_model.classifier[6] = torch.nn.Linear(MobileNetV3_model.classifier[6].in_features, classes_count)

# # Move the model to the device (GPU if available)
# MobileNetV3_model.to(device)

# # Define loss function and optimizer
# MobileNetV3_model_criterion = torch.nn.CrossEntropyLoss()
# MobileNetV3_model_optimizer = torch.optim.Adam(MobileNetV3_model.parameters(), lr=0.001)


In [ ]:
# summary(MobileNetV3_model, input_size=(3, 200, 200))


### ResNet 18


In [ ]:
ResNet18_model = models.resnet18()

# Freeze model parameters
for param in ResNet18_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
ResNet18_model.fc = torch.nn.Linear(ResNet18_model.fc.in_features, classes_count)

# Move the model to the device (GPU if available)
ResNet18_model.to(device)

# Define loss function and optimizer
ResNet18_model_criterion = torch.nn.CrossEntropyLoss()
ResNet18_model_optimizer = torch.optim.Adam(ResNet18_model.parameters(), lr=0.001)


In [ ]:
summary(ResNet18_model, input_size=(3, 200, 200))


### ResNet 50


In [ ]:
ResNet50_model = models.resnet50()

# Freeze model parameters
for param in ResNet50_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
ResNet50_model.fc = torch.nn.Linear(ResNet50_model.fc.in_features, classes_count)

# Move the model to the device (GPU if available)
ResNet50_model.to(device)

# Define loss function and optimizer
ResNet50_model_criterion = torch.nn.CrossEntropyLoss()
ResNet50_model_optimizer = torch.optim.Adam(ResNet50_model.parameters(), lr=0.001)


In [ ]:
summary(ResNet50_model, input_size=(3, 200, 200))


### Vgg16


In [ ]:
Vgg16_model = models.vgg16()

# Freeze model parameters
for param in Vgg16_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
Vgg16_model.classifier[6] = torch.nn.Linear(Vgg16_model.classifier[6].in_features, classes_count)

# Move the model to the device (GPU if available)
Vgg16_model.to(device)

# Define loss function and optimizer
Vgg16_model_criterion = torch.nn.CrossEntropyLoss()
Vgg16_model_optimizer = torch.optim.Adam(Vgg16_model.parameters(), lr=0.001)


In [ ]:
summary(Vgg16_model, input_size=(3, 200, 200))


### Vgg19


In [ ]:
Vgg19_model = models.vgg16()

# Freeze model parameters
for param in Vgg19_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
Vgg19_model.classifier[6] = torch.nn.Linear(Vgg19_model.classifier[6].in_features, classes_count)

# Move the model to the device (GPU if available)
Vgg19_model.to(device)

# Define loss function and optimizer
Vgg19_model_criterion = torch.nn.CrossEntropyLoss()
Vgg19_model_optimizer = torch.optim.Adam(Vgg19_model.parameters(), lr=0.001)


In [ ]:
summary(Vgg19_model, input_size=(3, 200, 200))


### Custom


## Train models


In [ ]:
# train model function
def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    epochs_losses = []
    i = 1
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
        epoch_loss /= len(train_loader.dataset)
        epochs_losses[i] = epoch_loss
        i = i + 1
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    return epochs_losses


### AlexNet


In [ ]:
model_path = 'tmp/models/ASL_AlexNet_model_V1.0.0.pth'
if not os.path.exists(model_path):
    AlexNet_model_losses = train_model(AlexNet_model, train_loader, AlexNet_model_criterion, AlexNet_model_optimizer, device, num_epochs=10)
    torch.save(AlexNet_model, model_path)


In [ ]:
AlexNet_model_losses


### GoogleNet


In [ ]:
model_path = 'tmp/models/ASL_GoogleNet_model_V1.0.0.pth'
if not os.path.exists(model_path):
    GoogleNet_model_losses = train_model(GoogleNet_model, train_loader, GoogleNet_model_criterion, GoogleNet_model_optimizer, device, num_epochs=10)
    torch.save(GoogleNet_model, model_path)


In [ ]:
GoogleNet_model_losses


### MobileNet V2


In [ ]:
# model_path = 'tmp/models/ASL_MobileNetV2_model_V1.0.0.pth'
# if not os.path.exists(model_path):
#     MobileNetV2_model_losses = train_model(MobileNetV2_model, train_loader, MobileNetV2_model_criterion, MobileNetV2_model_optimizer, device, num_epochs=10)
#     torch.save(MobileNetV2_model, model_path)


In [ ]:
# MobileNetV2_model_losses


### MobileNet V3


In [ ]:
# model_path = 'tmp/models/ASL_MobileNetV3_model_V1.0.0.pth'
# if not os.path.exists(model_path):
#     MobileNetV3_model_losses = train_model(MobileNetV3_model, train_loader, MobileNetV3_model_criterion, MobileNetV3_model_optimizer, device, num_epochs=10)
#     torch.save(MobileNetV3_model, model_path)


In [ ]:
# MobileNetV3_model_losses


### ResNet 18


In [ ]:
model_path = 'tmp/models/ASL_ResNet18_model_V1.0.0.pth'
if not os.path.exists(model_path):
    ResNet18_model_losses = train_model(ResNet18_model, train_loader, ResNet18_model_criterion, ResNet18_model_optimizer, device, num_epochs=10)
    torch.save(ResNet18_model, model_path)


In [ ]:
ResNet18_model_losses


### ResNet 50


In [ ]:
model_path = 'tmp/models/ASL_ResNet50_model_V1.0.0.pth'
if not os.path.exists(model_path):
    ResNet50_model_losses = train_model(ResNet50_model, train_loader, ResNet50_model_criterion, ResNet50_model_optimizer, device, num_epochs=10)
    torch.save(ResNet50_model, model_path)


In [ ]:
ResNet50_model_losses


### Vgg16


In [ ]:
model_path = 'tmp/models/ASL_Vgg16_model_V1.0.0.pth'
if not os.path.exists(model_path):
    Vgg16_model_losses = train_model(Vgg16_model, train_loader, Vgg16_model_criterion, Vgg16_model_optimizer, device, num_epochs=10)
    torch.save(Vgg16_model, model_path)


In [ ]:
Vgg16_model_losses


### Vgg19


In [ ]:
model_path = 'tmp/models/ASL_Vgg19_model_V1.0.0.pth'
if not os.path.exists(model_path):
    Vgg19_model_losses = train_model(Vgg19_model, train_loader, Vgg19_model_criterion, Vgg19_model_optimizer, device, num_epochs=10)
    torch.save(Vgg19_model, model_path)


In [ ]:
Vgg19_model_losses


### Custom


In [ ]:
# model_path = 'tmp/models/ASL_Custom_model_V1.0.0.pth'
# if not os.path.exists(model_path):
#     Custom_model_losses = train_model(Custom_model, train_loader, Custom_model_criterion, Custom_model_optimizer, device, num_epochs=10)
#     torch.save(Custom_model, model_path)


In [ ]:
# Custom_model_losses


## Evaluate


### AlexNet


### GoogleNet


### MobileNet V2


### MobileNet V3


### ResNet 18


### ResNet 50


### Vgg16


### Vgg19


### Custom


## Test


### AlexNet


### GoogleNet


### MobileNet V2


### MobileNet V3


### ResNet 18


### ResNet 50


### Vgg16


### Vgg19


### Custom
